In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm          # Importing statsmodels for linear model 
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score as rsq
#Random Forest Regrssor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# for integer encoding using feature-engine
from feature_engine.categorical_encoders import OrdinalCategoricalEncoder
#for XGBOOST 
import xgboost as xgb

In [2]:
# reading the data
data=pd.read_csv(r"C:\Users\sak\Desktop\excelr_proj_1\data_no_null.csv",encoding='latin1')# reading the data

In [3]:
data.head()

,Unnamed: 0,loan_amnt,terms,Rate_of_intrst,grade,sub_grade,home_ownership,annual_inc,verification_status,purpose,...,collection_recovery_fee,collections_12_mths_ex_med,application_type,last_week_pay,acc_now_delinq,Emp_designation,Experience,mths_since_last_delinq,tot_curr_bal,tot_colle_amt
0,0,14350,36 months,19.19,E,E3,OWN,28700.0,Source Verified,debt_consolidation,...,0.0,0.0,INDIVIDUAL,26th week,0.0,clerk,9 years,50.0,28699.0,0.0
1,1,4800,36 months,10.99,B,B4,MORTGAGE,65000.0,Source Verified,home_improvement,...,0.0,0.0,INDIVIDUAL,9th week,0.0,Human Resources Specialist,< 1 year,31.0,9974.0,0.0
2,2,10000,36 months,7.26,A,A4,OWN,45000.0,Not Verified,debt_consolidation,...,0.0,0.0,INDIVIDUAL,9th week,0.0,Driver,2 years,34.0,38295.0,65.0
3,3,15000,36 months,19.72,D,D5,RENT,105000.0,Not Verified,debt_consolidation,...,0.0,0.0,INDIVIDUAL,135th week,0.0,Us office of Personnel Management,10+ years,46.0,55564.0,0.0
4,4,16000,36 months,10.64,B,B2,RENT,52000.0,Verified,credit_card,...,0.0,0.0,INDIVIDUAL,96th week,0.0,LAUSD-HOLLYWOOD HIGH SCHOOL,10+ years,31.0,47159.0,0.0


In [4]:
data.drop(columns='Unnamed: 0',inplace=True,axis=1)

In [5]:
data

,loan_amnt,terms,Rate_of_intrst,grade,sub_grade,home_ownership,annual_inc,verification_status,purpose,State,...,collection_recovery_fee,collections_12_mths_ex_med,application_type,last_week_pay,acc_now_delinq,Emp_designation,Experience,mths_since_last_delinq,tot_curr_bal,tot_colle_amt
0,14350,36 months,19.19,E,E3,OWN,28700.0,Source Verified,debt_consolidation,FL,...,0.0,0.0,INDIVIDUAL,26th week,0.0,clerk,9 years,50.0,28699.0,0.0
1,4800,36 months,10.99,B,B4,MORTGAGE,65000.0,Source Verified,home_improvement,MD,...,0.0,0.0,INDIVIDUAL,9th week,0.0,Human Resources Specialist,< 1 year,31.0,9974.0,0.0
2,10000,36 months,7.26,A,A4,OWN,45000.0,Not Verified,debt_consolidation,OH,...,0.0,0.0,INDIVIDUAL,9th week,0.0,Driver,2 years,34.0,38295.0,65.0
3,15000,36 months,19.72,D,D5,RENT,105000.0,Not Verified,debt_consolidation,VA,...,0.0,0.0,INDIVIDUAL,135th week,0.0,Us office of Personnel Management,10+ years,46.0,55564.0,0.0
4,16000,36 months,10.64,B,B2,RENT,52000.0,Verified,credit_card,CA,...,0.0,0.0,INDIVIDUAL,96th week,0.0,LAUSD-HOLLYWOOD HIGH SCHOOL,10+ years,31.0,47159.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,15000,36 months,6.49,A,A2,MORTGAGE,97250.0,Not Verified,credit_card,TX,...,0.0,0.0,INDIVIDUAL,83th week,0.0,Network administrator,1 year,77.0,143920.0,0.0
887375,35000,60 months,25.89,G,G3,OWN,180000.0,Verified,debt_consolidation,NY,...,0.0,0.0,INDIVIDUAL,39th week,0.0,Credit Risk Analyst,1 year,19.0,772713.0,0.0
887376,14000,60 months,17.51,E,E4,OWN,56000.0,Verified,home_improvement,NH,...,0.0,0.0,INDIVIDUAL,87th week,0.0,USPS/Nashua L&DC,10+ years,54.0,71973.5,0.0
887377,20000,36 months,7.26,A,A4,MORTGAGE,100000.0,Source Verified,credit_card,MD,...,0.0,0.0,INDIVIDUAL,35th week,0.0,Computer Engineer,< 1 year,33.0,291998.0,0.0


# preprocessing and data cleaning 

### droping some unnecessary columns 

deletion1:- here i m dropping subgrade column because its cardinality is high as well as if i will group them then it will give similar column which i already have as Grade 

deletion2:- dropping state columns because it have ['FL', 'MD', 'OH', ..., 'NH', 'MD', 'FL'] these types of values which also not giving any infrmation about the states so irrelevent for user input 

deletion3:-i m deleting this EMP_designation column because it is having high cardinality and not much information to group it , and make it low cardinality 

deletion4 :- i m deleting this last_week_pay column because it is having high cardinality and not much information to group it , and make it low cardinality



In [6]:
# list of columns not necessary for prediction
cols_to_drop=['sub_grade','State','Emp_designation','last_week_pay']
# dropping the unnecessary columns
data_1=data.drop(columns=cols_to_drop,axis=1)

In [7]:
data_1.columns

Index(['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt'],
      dtype='object')

In [8]:
categorical = [col for col in data_1.columns if data_1[col].dtypes == 'O']

categorical

['terms',
 'grade',
 'home_ownership',
 'verification_status',
 'purpose',
 'initial_list_status',
 'application_type',
 'Experience']

In [9]:
data_1

,loan_amnt,terms,Rate_of_intrst,grade,home_ownership,annual_inc,verification_status,purpose,debt_income_ratio,delinq_2yrs,...,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,application_type,acc_now_delinq,Experience,mths_since_last_delinq,tot_curr_bal,tot_colle_amt
0,14350,36 months,19.19,E,OWN,28700.0,Source Verified,debt_consolidation,33.88,0.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,9 years,50.0,28699.0,0.0
1,4800,36 months,10.99,B,MORTGAGE,65000.0,Source Verified,home_improvement,3.64,0.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,< 1 year,31.0,9974.0,0.0
2,10000,36 months,7.26,A,OWN,45000.0,Not Verified,debt_consolidation,18.42,0.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,2 years,34.0,38295.0,65.0
3,15000,36 months,19.72,D,RENT,105000.0,Not Verified,debt_consolidation,14.97,0.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,10+ years,46.0,55564.0,0.0
4,16000,36 months,10.64,B,RENT,52000.0,Verified,credit_card,20.16,0.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,10+ years,31.0,47159.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,15000,36 months,6.49,A,MORTGAGE,97250.0,Not Verified,credit_card,12.46,0.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,1 year,77.0,143920.0,0.0
887375,35000,60 months,25.89,G,OWN,180000.0,Verified,debt_consolidation,11.78,2.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,1 year,19.0,772713.0,0.0
887376,14000,60 months,17.51,E,OWN,56000.0,Verified,home_improvement,16.44,0.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,10+ years,54.0,71973.5,0.0
887377,20000,36 months,7.26,A,MORTGAGE,100000.0,Source Verified,credit_card,15.00,0.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,< 1 year,33.0,291998.0,0.0


## Here , i m making 'purpose' column as low cardinality by grouping labels to similar labels
exaple:-

        lable1:-debt_consolidation                as debt_consolidation

        label2:-credit_card                       as credit_card
        
        label3:-home_improvement                  as home_improvement
        
        lable4:-major_purchase+car+house          as major_purchase
        
        lable5:-small_business+renewable_energy   as small_business
        
        label6:-weddings+others+vacations+moving  as others
        
        label7:-medical+educations                as medical(or)education


In [10]:
data.purpose.value_counts().sort_values()

educational              423
renewable_energy         575
wedding                 2347
house                   3707
vacation                4736
moving                  5414
medical                 8540
car                     8863
small_business         10377
major_purchase         17277
other                  42894
home_improvement       51829
credit_card           206182
debt_consolidation    524215
Name: purpose, dtype: int64

In [11]:
#df[['A','B']] = df[['A','B']].replace([1, 3, 2], [3, 6, 7])
data_1[['purpose']] = data_1[['purpose']].replace(['car','house','renewable_energy','wedding','vacation','moving','medical','educational'],
                                              ['major_purchase','major_purchase','small_business','other','other','other','medical(or)education','medical(or)education'])

In [12]:
data_1.purpose.value_counts().sort_values()

medical(or)education      8963
small_business           10952
major_purchase           29847
home_improvement         51829
other                    55391
credit_card             206182
debt_consolidation      524215
Name: purpose, dtype: int64

In [13]:
data_1.application_type.value_counts().sort_values()

JOINT            511
INDIVIDUAL    886868
Name: application_type, dtype: int64

## Train-Test split 

In [14]:
data_1.columns

Index(['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt'],
      dtype='object')

In [15]:
# to split the datasets
from sklearn.model_selection import train_test_split

# for one hot encoding with feature-engine
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

In [16]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data_1[['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership','annual_inc', 'verification_status',
            'purpose', 'debt_income_ratio','delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec','total_credits', 
            'initial_list_status','total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 
            'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq', 'Experience', 'mths_since_last_delinq',
            'tot_curr_bal', 'tot_colle_amt']],  # predictors
    data_1[['total revol_bal']],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((621165, 26), (266214, 26))

In [17]:
categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']

categorical

['terms',
 'grade',
 'home_ownership',
 'verification_status',
 'purpose',
 'initial_list_status',
 'application_type',
 'Experience']

In [18]:
ohe_enc = OneHotCategoricalEncoder(
    variables=categorical,
    drop_last=False)

ohe_enc.fit(X_train)

OneHotCategoricalEncoder(drop_last=False, top_categories=None,
                         variables=['terms', 'grade', 'home_ownership',
                                    'verification_status', 'purpose',
                                    'initial_list_status', 'application_type',
                                    'Experience'])

In [19]:
# in the encoder dict we can observe each of the top categories
# selected for each of the variables

ohe_enc.encoder_dict_

{'terms': array(['60 months', '36 months'], dtype=object),
 'grade': array(['D', 'C', 'A', 'B', 'F', 'E', 'G'], dtype=object),
 'home_ownership': array(['MORTGAGE', 'RENT', 'OWN', 'OTHER', 'NONE', 'ANY'], dtype=object),
 'verification_status': array(['Verified', 'Source Verified', 'Not Verified'], dtype=object),
 'purpose': array(['debt_consolidation', 'credit_card', 'home_improvement',
        'small_business', 'other', 'major_purchase',
        'medical(or)education'], dtype=object),
 'initial_list_status': array(['w', 'f'], dtype=object),
 'application_type': array(['INDIVIDUAL', 'JOINT'], dtype=object),
 'Experience': array(['10+ years', '2 years', '4 years', '< 1 year', '3 years',
        '7 years', '8 years', '1 year', '6 years', '5 years', '9 years'],
       dtype=object)}

In [20]:
X_train = ohe_enc.transform(X_train)
X_test = ohe_enc.transform(X_test)

# let's explore the result
X_train.head()

,loan_amnt,Rate_of_intrst,annual_inc,debt_income_ratio,delinq_2yrs,inq_last_6mths,numb_credit,pub_rec,total_credits,total_rec_int,...,Experience_2 years,Experience_4 years,Experience_< 1 year,Experience_3 years,Experience_7 years,Experience_8 years,Experience_1 year,Experience_6 years,Experience_5 years,Experience_9 years
416835,35000,16.55,125000.0,25.83,0.0,3.0,19.0,0.0,52.0,911.92,...,0,0,0,0,0,0,0,0,0,0
596073,10000,12.29,24000.0,13.70,0.0,1.0,6.0,0.0,27.0,901.03,...,1,0,0,0,0,0,0,0,0,0
759330,8650,8.94,30000.0,11.80,0.0,0.0,5.0,0.0,21.0,710.73,...,0,1,0,0,0,0,0,0,0,0
398838,5000,11.53,66000.0,34.45,0.0,1.0,15.0,0.0,22.0,384.60,...,0,0,0,0,0,0,0,0,0,0
401686,12000,9.99,84000.0,10.31,0.0,0.0,8.0,4.0,12.0,932.96,...,0,0,1,0,0,0,0,0,0,0


In [21]:
X_test.head()

,loan_amnt,Rate_of_intrst,annual_inc,debt_income_ratio,delinq_2yrs,inq_last_6mths,numb_credit,pub_rec,total_credits,total_rec_int,...,Experience_2 years,Experience_4 years,Experience_< 1 year,Experience_3 years,Experience_7 years,Experience_8 years,Experience_1 year,Experience_6 years,Experience_5 years,Experience_9 years
372989,10000,13.99,47000.0,12.69,0.0,0.0,14.0,0.0,15.0,754.43,...,0,0,0,0,0,0,0,0,0,0
9769,18000,12.99,60000.0,19.66,0.0,0.0,9.0,0.0,23.0,6287.44,...,0,0,0,1,0,0,0,0,0,0
134461,6000,15.10,50000.0,28.01,1.0,1.0,11.0,0.0,19.0,1333.97,...,0,0,0,0,0,0,0,0,0,0
425772,12950,14.99,36899.0,15.88,0.0,2.0,7.0,0.0,21.0,2125.63,...,0,0,1,0,0,0,0,0,0,0
409080,1900,9.99,42000.0,23.06,0.0,1.0,11.0,0.0,22.0,148.69,...,0,0,1,0,0,0,0,0,0,0


In [22]:
d=data_1.iloc[[0]]
d_1=d[['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership','annual_inc', 'verification_status',
            'purpose', 'debt_income_ratio','delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec','total_credits', 
            'initial_list_status','total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 
            'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq', 'Experience', 'mths_since_last_delinq',
            'tot_curr_bal', 'tot_colle_amt']]

In [23]:
import pickle


In [24]:
'''#saving the model to the local file system
filename = 'finalized_model.pickle'
pickle.dump(xgb_clf, open(filename, 'wb'))'''


"#saving the model to the local file system\nfilename = 'finalized_model.pickle'\npickle.dump(xgb_clf, open(filename, 'wb'))"

In [25]:
'''filehandler = encoder.pickle
pickle.dump(ohe_enc,open(filehandler,'wb'))'''

"filehandler = encoder.pickle\npickle.dump(ohe_enc,open(filehandler,'wb'))"

In [26]:
from sklearn.externals import joblib
joblib.dump(ohe_enc , 'model.pkl')

C:\Users\sak\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['model.pkl']

In [27]:
encoder = joblib.load('model.pkl')


In [28]:
#X_test = ohe_enc.transform(X_test)
d_test = encoder.transform(d_1)

In [29]:
d_1

,loan_amnt,terms,Rate_of_intrst,grade,home_ownership,annual_inc,verification_status,purpose,debt_income_ratio,delinq_2yrs,...,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,application_type,acc_now_delinq,Experience,mths_since_last_delinq,tot_curr_bal,tot_colle_amt
0,14350,36 months,19.19,E,OWN,28700.0,Source Verified,debt_consolidation,33.88,0.0,...,0.0,0.0,0.0,0.0,INDIVIDUAL,0.0,9 years,50.0,28699.0,0.0


In [30]:
d_test

,loan_amnt,Rate_of_intrst,annual_inc,debt_income_ratio,delinq_2yrs,inq_last_6mths,numb_credit,pub_rec,total_credits,total_rec_int,...,Experience_2 years,Experience_4 years,Experience_< 1 year,Experience_3 years,Experience_7 years,Experience_8 years,Experience_1 year,Experience_6 years,Experience_5 years,Experience_9 years
0,14350,19.19,28700.0,33.88,0.0,1.0,14.0,1.0,28.0,1173.84,...,0,0,0,0,0,0,0,0,0,1


# model analysis 

### scaling

In [31]:
''''# the scaler - for standardisation
from sklearn.preprocessing import StandardScaler'''

"'# the scaler - for standardisation\nfrom sklearn.preprocessing import StandardScaler"

In [32]:
'''# standardisation: with the StandardScaler from sklearn

# set up the scaler
scaler = StandardScaler()

# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)

# transform train and test sets
X_train_scaled = scaler.transform(X_train)
#X_test_scaled = scaler.transform(X_test)'''

'# standardisation: with the StandardScaler from sklearn\n\n# set up the scaler\nscaler = StandardScaler()\n\n# fit the scaler to the train set, it will learn the parameters\nscaler.fit(X_train)\n\n# transform train and test sets\nX_train_scaled = scaler.transform(X_train)\n#X_test_scaled = scaler.transform(X_test)'

In [33]:
#X_train_scaled

In [34]:
#joblib.dump(scaler , 'scaler.pkl')

In [35]:
#scaler = joblib.load('scaler.pkl')

In [36]:
#X_test_scaled = scaler.transform(X_test)

In [37]:
#X_test_scaled

In [38]:
#defining a normalisation function 
def normalize (x): 
    return ( (x-np.min(x))/ (max(x) - min(x)))
                                            
                                              
# applying normalize ( ) to all columns 
X_train = X_train.apply(normalize) 
y_train = y_train.apply(normalize)

### model building 

In [39]:
import statsmodels.api as sm          # Importing statsmodels
X_train = sm.add_constant(X_train)    # Adding a constant column to our dataframe
# create a first fitted model
lm_1 = sm.OLS(y_train,X_train).fit()

In [40]:
#Let's see the summary of our first linear model
print(lm_1.summary())

                            OLS Regression Results                            
Dep. Variable:        total revol_bal   R-squared:                       0.268
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     4364.
Date:                Thu, 30 Jul 2020   Prob (F-statistic):               0.00
Time:                        18:44:56   Log-Likelihood:             2.2257e+06
No. Observations:              621165   AIC:                        -4.451e+06
Df Residuals:                  621112   BIC:                        -4.451e+06
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [42]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [43]:
print(regressor.summary())

AttributeError: 'LinearRegression' object has no attribute 'summary'

##############################################################################################################################

In [2]:
# reading the data
df=pd.read_csv(r"C:\Users\sak\Desktop\excelr_proj_1\data_no_null.csv",encoding='latin1')# reading the data

In [3]:
df.head()

,Unnamed: 0,loan_amnt,terms,Rate_of_intrst,grade,sub_grade,home_ownership,annual_inc,verification_status,purpose,...,collection_recovery_fee,collections_12_mths_ex_med,application_type,last_week_pay,acc_now_delinq,Emp_designation,Experience,mths_since_last_delinq,tot_curr_bal,tot_colle_amt
0,0,14350,36 months,19.19,E,E3,OWN,28700.0,Source Verified,debt_consolidation,...,0.0,0.0,INDIVIDUAL,26th week,0.0,clerk,9 years,50.0,28699.0,0.0
1,1,4800,36 months,10.99,B,B4,MORTGAGE,65000.0,Source Verified,home_improvement,...,0.0,0.0,INDIVIDUAL,9th week,0.0,Human Resources Specialist,< 1 year,31.0,9974.0,0.0
2,2,10000,36 months,7.26,A,A4,OWN,45000.0,Not Verified,debt_consolidation,...,0.0,0.0,INDIVIDUAL,9th week,0.0,Driver,2 years,34.0,38295.0,65.0
3,3,15000,36 months,19.72,D,D5,RENT,105000.0,Not Verified,debt_consolidation,...,0.0,0.0,INDIVIDUAL,135th week,0.0,Us office of Personnel Management,10+ years,46.0,55564.0,0.0
4,4,16000,36 months,10.64,B,B2,RENT,52000.0,Verified,credit_card,...,0.0,0.0,INDIVIDUAL,96th week,0.0,LAUSD-HOLLYWOOD HIGH SCHOOL,10+ years,31.0,47159.0,0.0


In [4]:
df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [5]:
categorical = [col for col in df.columns if df[col].dtypes == 'O']

categorical

['terms',
 'grade',
 'sub_grade',
 'home_ownership',
 'verification_status',
 'purpose',
 'State',
 'initial_list_status',
 'application_type',
 'last_week_pay',
 'Emp_designation',
 'Experience']

In [7]:
df_1=df.drop(columns=categorical,axis=1)

In [8]:
df_1.columns

Index(['loan_amnt ', 'Rate_of_intrst', 'annual_inc', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'acc_now_delinq',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt'],
      dtype='object')

In [9]:
#defining a normalisation function 
def normalize (x): 
    return ( (x-np.min(x))/ (max(x) - min(x)))

df_1 = df_1.apply(normalize)

In [10]:
df_1

,loan_amnt,Rate_of_intrst,annual_inc,debt_income_ratio,delinq_2yrs,inq_last_6mths,numb_credit,pub_rec,total revol_bal,total_credits,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,acc_now_delinq,mths_since_last_delinq,tot_curr_bal,tot_colle_amt
0,0.401449,0.585974,0.003021,0.003388,0.000000,0.030303,0.155556,0.011628,0.007751,0.160714,0.048495,0.0,0.0,0.0,0.0,0.0,0.265957,0.003587,0.000000
1,0.124638,0.239544,0.006842,0.000364,0.000000,0.030303,0.066667,0.000000,0.002625,0.071429,0.003468,0.0,0.0,0.0,0.0,0.0,0.164894,0.001247,0.000000
2,0.275362,0.081960,0.004737,0.001842,0.000000,0.000000,0.055556,0.000000,0.003744,0.107143,0.002333,0.0,0.0,0.0,0.0,0.0,0.180851,0.004787,0.000007
3,0.420290,0.608365,0.011053,0.001497,0.000000,0.060606,0.111111,0.000000,0.004720,0.119048,0.200723,0.0,0.0,0.0,0.0,0.0,0.244681,0.006945,0.000000
4,0.449275,0.224757,0.005474,0.002016,0.000000,0.000000,0.122222,0.000000,0.012336,0.154762,0.094871,0.0,0.0,0.0,0.0,0.0,0.164894,0.005895,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,0.420290,0.049430,0.010237,0.001246,0.000000,0.000000,0.133333,0.000000,0.004200,0.136905,0.048762,0.0,0.0,0.0,0.0,0.0,0.409574,0.017990,0.000000
887375,1.000000,0.869033,0.018947,0.001178,0.051282,0.060606,0.233333,0.000000,0.018102,0.190476,0.268880,0.0,0.0,0.0,0.0,0.0,0.101064,0.096588,0.000000
887376,0.391304,0.514998,0.005895,0.001644,0.000000,0.090909,0.122222,0.011628,0.003769,0.142857,0.142787,0.0,0.0,0.0,0.0,0.0,0.287234,0.008997,0.000000
887377,0.565217,0.081960,0.010526,0.001500,0.000000,0.000000,0.077778,0.000000,0.011567,0.059524,0.031680,0.0,0.0,0.0,0.0,0.0,0.175532,0.036499,0.000000


In [11]:
df_1.columns

Index(['loan_amnt ', 'Rate_of_intrst', 'annual_inc', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'acc_now_delinq',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt'],
      dtype='object')

In [12]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    df_1[['loan_amnt ', 'Rate_of_intrst', 'annual_inc', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
        'total_credits', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'acc_now_delinq',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt']],  # predictors
    df_1[['total revol_bal']],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((621165, 18), (266214, 18))

In [13]:
X_train = sm.add_constant(X_train)    # Adding a constant column to our dataframe
# create a first fitted model
lm_1 = sm.OLS(y_train,X_train).fit()

In [14]:
#Let's see the summary of our first linear model
print(lm_1.summary())

                            OLS Regression Results                            
Dep. Variable:        total revol_bal   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                 1.184e+04
Date:                Thu, 30 Jul 2020   Prob (F-statistic):               0.00
Time:                        20:06:10   Log-Likelihood:             2.2207e+06
No. Observations:              621165   AIC:                        -4.441e+06
Df Residuals:                  621146   BIC:                        -4.441e+06
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

###############################################################################################################################

In [2]:
# reading the data
df2=pd.read_csv(r"C:\Users\sak\Desktop\excelr_proj_1\data (1).csv",encoding='latin1')# reading the data

C:\Users\sak\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df2_a=df2.drop(columns=['member_id ','batch_ID ','verification_status_joint','mths_since_last_major_derog',
                        'mths_since_last_record','mths_since_last_delinq'],axis=1)

In [4]:
df2_a.columns

Index(['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'sub_grade',
       'Emp_designation', 'Experience', 'home_ownership', 'annual_inc',
       'verification_status', 'purpose', 'State', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'last_week_pay', 'acc_now_delinq', 'tot_colle_amt',
       'tot_curr_bal'],
      dtype='object')

In [5]:
categorical = [col for col in df2_a.columns if df2_a[col].dtypes == 'O']

categorical

['terms',
 'grade',
 'sub_grade',
 'Emp_designation',
 'Experience',
 'home_ownership',
 'verification_status',
 'purpose',
 'State',
 'initial_list_status',
 'application_type',
 'last_week_pay']

In [6]:
df2_b=df2_a.drop(columns=categorical,axis=1)

In [7]:
df2_b

,loan_amnt,Rate_of_intrst,annual_inc,debt_income_ratio,delinq_2yrs,inq_last_6mths,numb_credit,pub_rec,total revol_bal,total_credits,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,acc_now_delinq,tot_colle_amt,tot_curr_bal
0,14350,19.19,28700.0,33.88,0.0,1.0,14.0,1.0,22515.0,28.0,1173.84,0.0,0.0,0.0,0.0,0.0,0.0,28699.0
1,4800,10.99,65000.0,3.64,0.0,1.0,6.0,0.0,7624.0,13.0,83.95,0.0,0.0,0.0,0.0,0.0,0.0,9974.0
2,10000,7.26,45000.0,18.42,0.0,0.0,5.0,0.0,10877.0,19.0,56.47,0.0,0.0,0.0,0.0,0.0,65.0,38295.0
3,15000,19.72,105000.0,14.97,0.0,2.0,10.0,0.0,13712.0,21.0,4858.62,0.0,0.0,0.0,0.0,0.0,0.0,55564.0
4,16000,10.64,52000.0,20.16,0.0,0.0,11.0,0.0,35835.0,27.0,2296.41,0.0,0.0,0.0,0.0,0.0,0.0,47159.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,15000,6.49,97250.0,12.46,0.0,0.0,12.0,0.0,12201.0,24.0,1180.32,0.0,0.0,0.0,0.0,0.0,0.0,143920.0
887375,35000,25.89,180000.0,11.78,2.0,2.0,21.0,0.0,52584.0,33.0,6508.41,0.0,0.0,0.0,0.0,0.0,0.0,772713.0
887376,14000,17.51,56000.0,16.44,0.0,3.0,11.0,1.0,10948.0,25.0,3456.24,0.0,0.0,0.0,0.0,0.0,NaN,NaN
887377,20000,7.26,100000.0,15.00,0.0,0.0,7.0,0.0,33599.0,11.0,766.84,0.0,0.0,0.0,0.0,0.0,0.0,291998.0


In [8]:
df2_c = df2_b.dropna(how='any',axis=0) 

In [9]:
df2_c

,loan_amnt,Rate_of_intrst,annual_inc,debt_income_ratio,delinq_2yrs,inq_last_6mths,numb_credit,pub_rec,total revol_bal,total_credits,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,acc_now_delinq,tot_colle_amt,tot_curr_bal
0,14350,19.19,28700.0,33.88,0.0,1.0,14.0,1.0,22515.0,28.0,1173.84,0.0,0.0,0.0,0.0,0.0,0.0,28699.0
1,4800,10.99,65000.0,3.64,0.0,1.0,6.0,0.0,7624.0,13.0,83.95,0.0,0.0,0.0,0.0,0.0,0.0,9974.0
2,10000,7.26,45000.0,18.42,0.0,0.0,5.0,0.0,10877.0,19.0,56.47,0.0,0.0,0.0,0.0,0.0,65.0,38295.0
3,15000,19.72,105000.0,14.97,0.0,2.0,10.0,0.0,13712.0,21.0,4858.62,0.0,0.0,0.0,0.0,0.0,0.0,55564.0
4,16000,10.64,52000.0,20.16,0.0,0.0,11.0,0.0,35835.0,27.0,2296.41,0.0,0.0,0.0,0.0,0.0,0.0,47159.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887372,10800,20.99,41000.0,32.70,0.0,3.0,17.0,1.0,3569.0,49.0,163.72,0.0,0.0,0.0,0.0,0.0,0.0,56129.0
887373,14300,15.41,52000.0,19.30,1.0,0.0,15.0,0.0,9687.0,33.0,159.16,0.0,0.0,0.0,0.0,0.0,0.0,88713.0
887374,15000,6.49,97250.0,12.46,0.0,0.0,12.0,0.0,12201.0,24.0,1180.32,0.0,0.0,0.0,0.0,0.0,0.0,143920.0
887375,35000,25.89,180000.0,11.78,2.0,2.0,21.0,0.0,52584.0,33.0,6508.41,0.0,0.0,0.0,0.0,0.0,0.0,772713.0


In [10]:
'''#defining a normalisation function 
def normalize (x): 
    return ( (x-np.min(x))/ (max(x) - min(x)))

df2_c = df2_c.apply(normalize)'''

'#defining a normalisation function \ndef normalize (x): \n    return ( (x-np.min(x))/ (max(x) - min(x)))\n\ndf2_c = df2_c.apply(normalize)'

In [11]:
df2_c.columns

Index(['loan_amnt ', 'Rate_of_intrst', 'annual_inc', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_colle_amt',
       'tot_curr_bal'],
      dtype='object')

In [12]:
X=df2_c[['loan_amnt ', 'Rate_of_intrst', 'annual_inc', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
        'total_credits', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_colle_amt',
       'tot_curr_bal']]
Y=df2_c[['total revol_bal']]

In [13]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
Y = sc_y.fit_transform(Y)

In [14]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(X  ,  # predictors
                                                    Y,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((571972, 17), (245131, 17))

In [15]:
X_train = sm.add_constant(X_train)    # Adding a constant column to our dataframe
# create a first fitted model
lm_1 = sm.OLS(y_train,X_train).fit()

In [16]:
#Let's see the summary of our first linear model
print(lm_1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                 1.190e+04
Date:                Thu, 30 Jul 2020   Prob (F-statistic):               0.00
Time:                        22:43:51   Log-Likelihood:            -7.2964e+05
No. Observations:              571972   AIC:                         1.459e+06
Df Residuals:                  571954   BIC:                         1.460e+06
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0005      0.001     -0.429      0.6

In [17]:
y_test=pd.DataFrame(data=sc_y.inverse_transform(y_test),columns=['revolving_balance'])

In [18]:
#X_test_m6 = sm.add_constant(X_test)
X_test_cons = sm.add_constant(X_test)

In [19]:
prediction= sc_y.inverse_transform(lm_1.predict(X_test_cons))

In [20]:
X_test.shape

(245131, 17)

In [21]:
y_test.insert(loc=1,column='prediction',value=prediction)

In [22]:
y_test

,revolving_balance,prediction
0,12922.0,6125.405139
1,32085.0,25930.967800
2,2128.0,3479.010216
3,14591.0,36940.839223
4,7642.0,7813.622059
...,...,...
245126,12647.0,23654.191561
245127,20674.0,18357.236260
245128,14356.0,12725.177479
245129,7222.0,5642.225630


In [23]:
import numpy as np
from sklearn import metrics
print('RMSE :', np.sqrt(metrics.mean_squared_error(y_test['revolving_balance'], y_test['prediction'])))

RMSE : 18947.630865761777


###############################################################################################################################

In [2]:
# reading the data ,data having no null values 
df3=pd.read_csv(r"C:\Users\sak\Desktop\excelr_proj_1\data_no_null.csv",encoding='latin1')# reading the data

In [3]:
# list of columns not necessary for prediction
cols_to_drop=['Unnamed: 0','sub_grade','State','Emp_designation','last_week_pay']
# dropping the unnecessary columns
df3_a=df3.drop(columns=cols_to_drop,axis=1)

In [4]:
categorical = [col for col in df3_a.columns if df3_a[col].dtypes == 'O']

categorical

['terms',
 'grade',
 'home_ownership',
 'verification_status',
 'purpose',
 'initial_list_status',
 'application_type',
 'Experience']

In [5]:
#df[['A','B']] = df[['A','B']].replace([1, 3, 2], [3, 6, 7])
df3_a[['purpose']] = df3_a[['purpose']].replace(['car','house','renewable_energy','wedding','vacation','moving','medical','educational'],
                                              ['major_purchase','major_purchase','small_business','other','other','other','medical(or)education','medical(or)education'])

In [6]:
df3_a.purpose.value_counts().sort_values()

medical(or)education      8963
small_business           10952
major_purchase           29847
home_improvement         51829
other                    55391
credit_card             206182
debt_consolidation      524215
Name: purpose, dtype: int64

In [7]:
from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder

In [8]:
count_enc = CountFrequencyCategoricalEncoder(
    encoding_method='frequency', # to do frequency ==> encoding_method='frequency'
    variables=categorical)

df3_encoded = count_enc.fit_transform(df3_a)

In [9]:
count_enc.encoder_dict_

{'terms': {'36 months': 0.699954585357553, '60 months': 0.30004541464244705},
 'grade': {'B': 0.2868391070782608,
  'C': 0.27706312635300134,
  'A': 0.16701093895618446,
  'D': 0.15725186194399463,
  'E': 0.07967846883913186,
  'F': 0.025970864760153214,
  'G': 0.00618563206927367},
 'home_ownership': {'MORTGAGE': 0.49985068386788506,
  'RENT': 0.4013133058140885,
  'OWN': 0.09857118547993586,
  'OTHER': 0.00020509838524463618,
  'NONE': 5.6345710232042906e-05,
  'ANY': 3.3807426139225743e-06},
 'verification_status': {'Source Verified': 0.3713835914530319,
  'Verified': 0.3280120444590192,
  'Not Verified': 0.3006043640879489},
 'purpose': {'debt_consolidation': 0.5907453297858074,
  'credit_card': 0.2323494245412614,
  'other': 0.06242090470926177,
  'home_improvement': 0.05840683631233103,
  'major_purchase': 0.03363500826591569,
  'small_business': 0.012341964369226677,
  'medical(or)education': 0.01010053201619601},
 'initial_list_status': {'f': 0.5148285005617668, 'w': 0.48517149

In [11]:
df3_encoded.columns

Index(['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt'],
      dtype='object')

In [12]:
X=df3_encoded[['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
        'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt']]
Y=df3_encoded[['total revol_bal']]

In [13]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
Y = sc_y.fit_transform(Y)

In [14]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(X  ,  # predictors
                                                    Y,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((621165, 26), (266214, 26))

In [15]:
X_train = sm.add_constant(X_train)    # Adding a constant column to our dataframe
# create a first fitted model
lm_1 = sm.OLS(y_train,X_train).fit()

In [16]:
#Let's see the summary of our first linear model
print(lm_1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     8451.
Date:                Fri, 31 Jul 2020   Prob (F-statistic):               0.00
Time:                        13:28:14   Log-Likelihood:            -7.9818e+05
No. Observations:              621165   AIC:                         1.596e+06
Df Residuals:                  621138   BIC:                         1.597e+06
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0004      0.001     -0.343      0.7

In [17]:
y_test=pd.DataFrame(data=sc_y.inverse_transform(y_test),columns=['revolving_balance'])

In [18]:
#X_test_m6 = sm.add_constant(X_test)
X_test_cons = sm.add_constant(X_test)

In [19]:
prediction= sc_y.inverse_transform(lm_1.predict(X_test_cons))

In [20]:
X_test.shape

(266214, 26)

In [21]:
y_test.insert(loc=1,column='prediction',value=prediction)

In [22]:
y_test

,revolving_balance,prediction
0,6068.0,14675.613812
1,22107.0,17232.122039
2,9951.0,10933.655428
3,5218.0,5433.189591
4,11218.0,6636.362253
...,...,...
266209,35409.0,30710.814432
266210,7277.0,20372.199458
266211,25770.0,39793.899857
266212,5775.0,9209.676005


In [23]:
import numpy as np
from sklearn import metrics
print('RMSE :', np.sqrt(metrics.mean_squared_error(y_test['revolving_balance'], y_test['prediction'])))

RMSE : 18851.730815790033


#######################################RANDOM FOREST REGRESSOR#################################

In [10]:
df3_encoded

,loan_amnt,terms,Rate_of_intrst,grade,home_ownership,annual_inc,verification_status,purpose,debt_income_ratio,delinq_2yrs,...,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,application_type,acc_now_delinq,Experience,mths_since_last_delinq,tot_curr_bal,tot_colle_amt
0,14350,0.699955,19.19,0.079678,0.098571,28700.0,0.371384,0.590745,33.88,0.0,...,0.0,0.0,0.0,0.0,0.999424,0.0,0.039055,50.0,28699.0,0.0
1,4800,0.699955,10.99,0.286839,0.499851,65000.0,0.371384,0.058407,3.64,0.0,...,0.0,0.0,0.0,0.0,0.999424,0.0,0.079566,31.0,9974.0,0.0
2,10000,0.699955,7.26,0.167011,0.098571,45000.0,0.300604,0.590745,18.42,0.0,...,0.0,0.0,0.0,0.0,0.999424,0.0,0.088880,34.0,38295.0,65.0
3,15000,0.699955,19.72,0.157252,0.401313,105000.0,0.300604,0.590745,14.97,0.0,...,0.0,0.0,0.0,0.0,0.999424,0.0,0.379087,46.0,55564.0,0.0
4,16000,0.699955,10.64,0.286839,0.401313,52000.0,0.328012,0.232349,20.16,0.0,...,0.0,0.0,0.0,0.0,0.999424,0.0,0.379087,31.0,47159.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,15000,0.699955,6.49,0.167011,0.499851,97250.0,0.300604,0.232349,12.46,0.0,...,0.0,0.0,0.0,0.0,0.999424,0.0,0.064341,77.0,143920.0,0.0
887375,35000,0.300045,25.89,0.006186,0.098571,180000.0,0.328012,0.590745,11.78,2.0,...,0.0,0.0,0.0,0.0,0.999424,0.0,0.064341,19.0,772713.0,0.0
887376,14000,0.300045,17.51,0.079678,0.098571,56000.0,0.328012,0.058407,16.44,0.0,...,0.0,0.0,0.0,0.0,0.999424,0.0,0.379087,54.0,71973.5,0.0
887377,20000,0.699955,7.26,0.167011,0.499851,100000.0,0.371384,0.232349,15.00,0.0,...,0.0,0.0,0.0,0.0,0.999424,0.0,0.079566,33.0,291998.0,0.0


In [11]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score as rsq
#Random Forest Regrssor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [12]:
df3_encoded.columns

Index(['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt'],
      dtype='object')

In [13]:
categorical

['terms',
 'grade',
 'home_ownership',
 'verification_status',
 'purpose',
 'initial_list_status',
 'application_type',
 'Experience']

In [14]:
X_c=df3_encoded[['loan_amnt ',  'Rate_of_intrst', 
       'annual_inc',   'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
        'total_credits', 
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
        'acc_now_delinq',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt']]
Y_c = df3_encoded[['total revol_bal']]

In [15]:
# let's separate into training and testing set

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_c  ,  # predictors
                                                    Y_c,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train_c.shape, X_test_c.shape

((621165, 18), (266214, 18))

In [16]:
#'''params_rand={
#        'n_estimators':[100,200,500,1000],
#        'max_depth':[10,15,20],
#        'max_features':['auto', 'sqrt', 'log2'],
#        'random_state':[42]
#         }'''

In [17]:
#'''rf_model=RandomForestRegressor()
#grid_search_rf = RandomizedSearchCV(rf_model,param_distributions=params_rand,n_jobs=-1,cv=5)
#grid_search_rf.fit(X_train_c,y_train_c)
#grid_search_rf.best_estimator_
#grid_search_rf.best_params_'''

In [31]:
rf_model = RandomForestRegressor(n_estimators=20, max_depth=15, random_state=42)
rf_model.fit(X_train_c,y_train_c)

C:\Users\sak\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=15, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=20, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [32]:
rf_pred_train=rf_model.predict(X_train_c) 

In [33]:
rf_train_r2=rsq(y_train_c,rf_pred_train)            
rf_train_r2

0.6605812572683716

In [34]:
rf_pred_test=rf_model.predict(X_test_c)
rf_test_r2=rsq(y_test_c,rf_pred_test)
rf_test_r2

0.3363033497002529

In [35]:
import numpy as np
from sklearn import metrics
print(' train RMSE :', np.sqrt(metrics.mean_squared_error(y_train_c,rf_pred_train)))
print(' test RMSE :', np.sqrt(metrics.mean_squared_error(y_test_c,rf_pred_test)))

 train RMSE : 13296.073241791735
 test RMSE : 17496.04589871282


In [53]:
k=list(rf_model.feature_importances_)
j=list(X_train_c.columns)
data ={'columns': j, 'feature importance':k}
d=pd.DataFrame(data)
d

,columns,feature importance
0,loan_amnt,0.117139
1,Rate_of_intrst,0.045207
2,annual_inc,0.104387
3,debt_income_ratio,0.133847
4,delinq_2yrs,0.009019
5,inq_last_6mths,0.011171
6,numb_credit,0.041301
7,pub_rec,0.016536
8,total_credits,0.044315
9,total_rec_int,0.036526


i m droping some of the columns whose importance is not good in our model 

In [54]:
X_c_1=df3_encoded[['loan_amnt ',  'Rate_of_intrst', 
       'annual_inc',   'debt_income_ratio',
        'inq_last_6mths', 'numb_credit', 'pub_rec',
        'total_credits', 
       'total_rec_int', 
      
        
       'mths_since_last_delinq', 'tot_curr_bal', ]]
Y_c_1 = df3_encoded[['total revol_bal']]

In [55]:
# let's separate into training and testing set

X_train_c_1, X_test_c_1, y_train_c_1, y_test_c_1 = train_test_split(X_c_1  ,  # predictors
                                                    Y_c_1,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train_c_1.shape, X_test_c_1.shape

((621165, 11), (266214, 11))

In [56]:
rf_model_1 = RandomForestRegressor(n_estimators=25, max_depth=20, random_state=42)
rf_model_1.fit(X_train_c_1,y_train_c_1)

C:\Users\sak\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=25, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [59]:
rf_pred_train_1=rf_model_1.predict(X_train_c_1) 
rf_train_r2_1=rsq(y_train_c_1,rf_pred_train_1)            
rf_train_r2_1

0.8044181347570576

In [60]:
rf_pred_test_1=rf_model_1.predict(X_test_c_1)
rf_test_r2_1=rsq(y_test_c_1,rf_pred_test_1)
rf_test_r2_1

0.3310314776925435

In [61]:
print(' train RMSE :', np.sqrt(metrics.mean_squared_error(y_train_c_1,rf_pred_train_1)))
print(' test RMSE :', np.sqrt(metrics.mean_squared_error(y_test_c_1,rf_pred_test_1)))

 train RMSE : 10092.988545513868
 test RMSE : 17565.395709719454


In [63]:
k_1=list(rf_model_1.feature_importances_)
j_1=list(X_train_c_1.columns)
data_1 ={'columns': j_1, 'feature importance':k_1}
d_1=pd.DataFrame(data_1)
d_1

,columns,feature importance
0,loan_amnt,0.112127
1,Rate_of_intrst,0.057767
2,annual_inc,0.107267
3,debt_income_ratio,0.136235
4,inq_last_6mths,0.019104
5,numb_credit,0.050778
6,pub_rec,0.014514
7,total_credits,0.057490
8,total_rec_int,0.058039
9,mths_since_last_delinq,0.057390


### including categorical columns 

In [66]:
df3_encoded.columns

Index(['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt'],
      dtype='object')

In [67]:
X_c_2=df3_encoded[['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
        'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt' ]]
Y_c_2 = df3_encoded[['total revol_bal']]

In [68]:
# let's separate into training and testing set

X_train_c_2, X_test_c_2, y_train_c_2, y_test_c_2 = train_test_split(X_c_2  ,  # predictors
                                                    Y_c_2,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train_c_2.shape, X_test_c_2.shape

((621165, 26), (266214, 26))

In [69]:
rf_model_2 = RandomForestRegressor(n_estimators=25, max_depth=20, random_state=42)
rf_model_2.fit(X_train_c_2,y_train_c_2)

C:\Users\sak\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=25, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [70]:
rf_pred_train_2=rf_model_2.predict(X_train_c_2) 
rf_train_r2_2=rsq(y_train_c_2,rf_pred_train_2)            
rf_train_r2_2

0.8056435305683445

In [71]:
rf_pred_test_2=rf_model_2.predict(X_test_c_2)
rf_test_r2_2=rsq(y_test_c_2,rf_pred_test_2)
rf_test_r2_2

0.33036533215426866

In [73]:
print(' train RMSE :', np.sqrt(metrics.mean_squared_error(y_train_c_2,rf_pred_train_2)))
print(' test RMSE :', np.sqrt(metrics.mean_squared_error(y_test_c_2,rf_pred_test_2)))

 train RMSE : 10061.320631867558
 test RMSE : 17574.13916931904


In [74]:
k_2=list(rf_model_2.feature_importances_)
j_2=list(X_train_c_2.columns)
data_2 ={'columns': j_2, 'feature importance':k_2}
d_2=pd.DataFrame(data_2)
d_2

,columns,feature importance
0,loan_amnt,0.104707
1,terms,0.004675
2,Rate_of_intrst,0.041357
3,grade,0.021650
4,home_ownership,0.011048
5,annual_inc,0.095597
6,verification_status,0.008963
7,purpose,0.020532
8,debt_income_ratio,0.121194
9,delinq_2yrs,0.007908


In [75]:
# i m dropping those columns which having importance less than 1 %

In [77]:
df3_encoded.columns

Index(['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt'],
      dtype='object')

In [88]:
X_c_2_a=df3_encoded[['loan_amnt ',  'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc',  'purpose', 'debt_income_ratio',
        'inq_last_6mths', 'numb_credit', 'pub_rec',
        'total_credits', 
       'total_rec_int', 
    
         'Experience',
       'mths_since_last_delinq', 'tot_curr_bal' ]]
Y_c_2_a = df3_encoded[['total revol_bal']]

In [89]:
# let's separate into training and testing set

X_train_c_2_a, X_test_c_2_a, y_train_c_2_a, y_test_c_2_a = train_test_split(X_c_2_a  ,  # predictors
                                                    Y_c_2_a,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train_c_2_a.shape, X_test_c_2_a.shape

((621165, 15), (266214, 15))

In [90]:
rf_model_2_a = RandomForestRegressor(n_estimators=25, max_depth=20, random_state=42)
rf_model_2_a.fit(X_train_c_2_a,y_train_c_2_a)

C:\Users\sak\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=25, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [91]:
rf_pred_train_2_a=rf_model_2_a.predict(X_train_c_2_a) 
rf_train_r2_2_a=rsq(y_train_c_2_a,rf_pred_train_2_a)            
rf_train_r2_2_a

0.8139923608270837

In [92]:
rf_pred_test_2_a=rf_model_2_a.predict(X_test_c_2_a)
rf_test_r2_2_a=rsq(y_test_c_2_a,rf_pred_test_2_a)
rf_test_r2_2_a

0.3261021451661832

In [93]:
print(' train RMSE :', np.sqrt(metrics.mean_squared_error(y_train_c_2_a,rf_pred_train_2_a)))
print(' test RMSE :', np.sqrt(metrics.mean_squared_error(y_test_c_2_a,rf_pred_test_2_a)))

 train RMSE : 9842.850275826091
 test RMSE : 17629.992739042245


In [94]:
k_2_a=list(rf_model_2_a.feature_importances_)
j_2_a=list(X_train_c_2_a.columns)
data_2_a ={'columns': j_2_a, 'feature importance':k_2_a}
d_2_a=pd.DataFrame(data_2_a)
d_2_a

,columns,feature importance
0,loan_amnt,0.107458
1,Rate_of_intrst,0.045165
2,grade,0.022324
3,home_ownership,0.013090
4,annual_inc,0.099578
5,purpose,0.022180
6,debt_income_ratio,0.125285
7,inq_last_6mths,0.017868
8,numb_credit,0.045621
9,pub_rec,0.012436


##   xgboost with label encoder 

In [2]:
# for integer encoding using feature-engine
from feature_engine.categorical_encoders import OrdinalCategoricalEncoder

In [3]:
# reading the data
data_xg=pd.read_csv(r"C:\Users\sak\Desktop\excelr_proj_1\data_no_null.csv",encoding='latin1')# reading the data

In [4]:
# list of columns not necessary for prediction
cols_to_drop=['Unnamed: 0','sub_grade','State','Emp_designation','last_week_pay']
# dropping the unnecessary columns
df_xg=data_xg.drop(columns=cols_to_drop,axis=1)

In [5]:
categorical = [col for col in df_xg.columns if df_xg[col].dtypes == 'O']

categorical

['terms',
 'grade',
 'home_ownership',
 'verification_status',
 'purpose',
 'initial_list_status',
 'application_type',
 'Experience']

In [6]:
#df[['A','B']] = df[['A','B']].replace([1, 3, 2], [3, 6, 7])
df_xg[['purpose']] = df_xg[['purpose']].replace(['car','house','renewable_energy','wedding','vacation','moving','medical','educational'],
                                              ['major_purchase','major_purchase','small_business','other','other','other','medical(or)education','medical(or)education'])

In [7]:
df_xg.purpose.value_counts().sort_values()

medical(or)education      8963
small_business           10952
major_purchase           29847
home_improvement         51829
other                    55391
credit_card             206182
debt_consolidation      524215
Name: purpose, dtype: int64

In [8]:
ordinal_enc = OrdinalCategoricalEncoder(
    encoding_method='arbitrary',
    variables=categorical)

df_xg_label_enc = ordinal_enc.fit_transform(df_xg)

In [9]:
#ordinal_enc.encoder_dict_

In [10]:
df_xg_label_enc.columns

Index(['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total revol_bal', 'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt'],
      dtype='object')

In [11]:
X_xg_l=df_xg_label_enc[['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt']]
Y_xg_l=df_xg_label_enc[['total revol_bal']]

In [113]:
# let's separate into training and testing set

X_train_xg_l, X_test_xg_l, y_train_xg_l, y_test_xg_l = train_test_split(X_xg_l  ,  # predictors
                                                    Y_xg_l,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train_xg_l.shape, X_test_xg_l.shape

((621165, 26), (266214, 26))

In [115]:
xgb_mod_l=xgb.XGBRegressor()
params_l={
        'learning_rate':[0.03,0.05,0.08,0.10,0.15,0.20,0.25,0.30],
        'max_depth':[3,4,5,6,8,10,12,15],
        'min_child_weight':[1,3,5,7],
        'gamma':[0.0,0.1,0.2,0.3,0.4]
        }
random_search=RandomizedSearchCV(xgb_mod_l,param_distributions=params_l,n_iter=5,n_jobs=-1,cv=5,verbose=3)
random_search.fit(X_train_xg_l,y_train_xg_l)
random_search.best_estimator_
random_search.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 42.8min finished


{'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.15, 'gamma': 0.4}

In [116]:
xgb_mod_l=xgb.XGBRegressor(min_child_weight=3, max_depth=6, learning_rate=0.15, gamma=0.4)
xgb_mod_l.fit(X_train_xg_l,y_train_xg_l)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.4, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.15, max_delta_step=0, max_depth=6,
             min_child_weight=3, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [117]:
xg_pred_train_l=xgb_mod_l.predict(X_train_xg_l) 
xg_train_l_r2=rsq(y_train_xg_l,xg_pred_train_l)            
xg_train_l_r2

0.5585968687194209

In [118]:
xg_pred_test_l=xgb_mod_l.predict(X_test_xg_l)
xg_test_l_r2=rsq(y_test_xg_l,xg_pred_test_l)
xg_test_l_r2

0.3553813892114036

In [119]:
print(' train RMSE :', np.sqrt(metrics.mean_squared_error(y_train_xg_l,xg_pred_train_l)))
print(' test RMSE :', np.sqrt(metrics.mean_squared_error(y_test_xg_l,xg_pred_test_l)))

 train RMSE : 15162.583076592067
 test RMSE : 17242.749401959194


In [134]:
k_xg_l=list(xgb_mod_l.feature_importances_)
j_xg_l=list(X_train_xg_l.columns)
data_xg_l ={'columns': j_xg_l, 'feature importance':k_xg_l}
d_xg_l=pd.DataFrame(data_xg_l)
d_xg_l

,columns,feature importance
0,loan_amnt,0.155720
1,terms,0.007756
2,Rate_of_intrst,0.017577
3,grade,0.181945
4,home_ownership,0.024229
5,annual_inc,0.056004
6,verification_status,0.014088
7,purpose,0.024618
8,debt_income_ratio,0.059872
9,delinq_2yrs,0.019557


In [124]:
comparision=y_test_xg_l.copy()
comparision.insert(loc=1,column='prediction',value=xg_pred_test_l)
comparision

,total revol_bal,prediction
372989,6068.0,10170.048828
9769,22107.0,16362.871094
134461,9951.0,12150.269531
425772,5218.0,7642.464844
409080,11218.0,9241.432617
...,...,...
695018,35409.0,21461.667969
822913,7277.0,13885.403320
120531,25770.0,29206.042969
486396,5775.0,9404.303711


## xg-boost with frequency encoder 

In [125]:
ordinal_enc = CountFrequencyCategoricalEncoder(
    encoding_method='frequency',
    variables=categorical)

df_xg_fre_enc = ordinal_enc.fit_transform(df_xg)

In [126]:
X_xg_f=df_xg_fre_enc[['loan_amnt ', 'terms', 'Rate_of_intrst', 'grade', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'debt_income_ratio',
       'delinq_2yrs', 'inq_last_6mths', 'numb_credit', 'pub_rec',
       'total_credits', 'initial_list_status',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'acc_now_delinq', 'Experience',
       'mths_since_last_delinq', 'tot_curr_bal', 'tot_colle_amt']]
Y_xg_f=df_xg_fre_enc[['total revol_bal']]

In [127]:
# let's separate into training and testing set

X_train_xg_f, X_test_xg_f, y_train_xg_f, y_test_xg_f = train_test_split(X_xg_f  ,  # predictors
                                                    Y_xg_f,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train_xg_f.shape, X_test_xg_f.shape

((621165, 26), (266214, 26))

In [128]:
xgb_mod_f=xgb.XGBRegressor()
params_f={
        'learning_rate':[0.03,0.05,0.08,0.10,0.15,0.20,0.25,0.30],
        'max_depth':[3,4,5,6,8,10,12,15],
        'min_child_weight':[1,3,5,7],
        'gamma':[0.0,0.1,0.2,0.3,0.4]
        }
random_search_f=RandomizedSearchCV(xgb_mod_f,param_distributions=params_f,n_iter=5,n_jobs=-1,cv=5,verbose=3)
random_search_f.fit(X_train_xg_f,y_train_xg_f)
random_search_f.best_estimator_
random_search_f.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 65.0min finished


{'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.08, 'gamma': 0.1}

In [129]:
xgb_mod_f=xgb.XGBRegressor(min_child_weight=5, max_depth=3, learning_rate=0.08, gamma=0.1)
xgb_mod_f.fit(X_train_xg_f,y_train_xg_f)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.08, max_delta_step=0, max_depth=3,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [130]:
xg_pred_train_f=xgb_mod_f.predict(X_train_xg_f) 
xg_train_f_r2=rsq(y_train_xg_f,xg_pred_train_f)            
xg_train_f_r2

0.437162562576662

In [131]:
xg_pred_test_f=xgb_mod_f.predict(X_test_xg_f)
xg_test_f_r2=rsq(y_test_xg_f,xg_pred_test_f)
xg_test_f_r2

0.35408539722943

In [132]:
print(' train RMSE :', np.sqrt(metrics.mean_squared_error(y_train_xg_f,xg_pred_train_f)))
print(' test RMSE :', np.sqrt(metrics.mean_squared_error(y_test_xg_f,xg_pred_test_f)))

 train RMSE : 17121.70369978368
 test RMSE : 17260.073788892674


In [133]:
k_xg_f=list(xgb_mod_f.feature_importances_)
j_xg_f=list(X_train_xg_f.columns)
data_xg_f ={'columns': j_xg_f, 'feature importance':k_xg_f}
d_xg_f=pd.DataFrame(data_xg_f)
d_xg_f

,columns,feature importance
0,loan_amnt,0.196949
1,terms,0.000000
2,Rate_of_intrst,0.013392
3,grade,0.092085
4,home_ownership,0.010979
5,annual_inc,0.110434
6,verification_status,0.004057
7,purpose,0.026032
8,debt_income_ratio,0.080987
9,delinq_2yrs,0.019744


In [135]:
comparision_f=y_test_xg_f.copy()
comparision_f.insert(loc=1,column='prediction',value=xg_pred_test_f)
comparision_f

,total revol_bal,prediction
372989,6068.0,11850.347656
9769,22107.0,17017.488281
134461,9951.0,11367.666016
425772,5218.0,7464.257324
409080,11218.0,10957.072266
...,...,...
695018,35409.0,21933.373047
822913,7277.0,14763.003906
120531,25770.0,29143.837891
486396,5775.0,10891.500000


## support vector machine on label encoding

In [12]:
X_sv_l=X_xg_l.copy()
Y_sv_l=Y_xg_l.copy()

In [13]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_sv_l = sc_X.fit_transform(X_sv_l)
Y_sv_l = sc_y.fit_transform(Y_sv_l)

In [14]:
X_sv_l

array([[-0.04804303, -0.65472446,  1.35633124, ...,  1.13685889,
        -0.72181207, -0.02100372],
       [-1.18016983, -0.65472446, -0.51501808, ..., -0.09610225,
        -0.8479816 , -0.02100372],
       [-0.56372382, -0.65472446, -1.36625381, ...,  0.09857582,
        -0.65715398, -0.01443463],
       ...,
       [-0.08953459,  1.52736008,  0.97293284, ...,  1.39642966,
        -0.43022739, -0.02100372],
       [ 0.62174927, -0.65472446, -1.36625381, ...,  0.03368313,
         1.0523033 , -0.02100372],
       [-1.42911918,  1.52736008, -1.31376474, ...,  0.09857582,
        -0.02591554, -0.02100372]])

In [15]:
# let's separate into training and testing set

X_train_sv_l, X_test_sv_l, y_train_sv_l, y_test_sv_l = train_test_split(X_sv_l  ,  # predictors
                                                    Y_sv_l,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train_sv_l.shape, X_test_sv_l.shape

((621165, 26), (266214, 26))

In [ ]:
from sklearn.svm import SVR
regressor_sv_l = SVR(kernel = 'linear')
regressor_sv_l.fit(X_train_sv_l, y_train_sv_l)

C:\Users\sak\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
